<a href="https://colab.research.google.com/github/Alex-Ukraine/ml-selfedu/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В 2014 году Ян Гудфеллов (Ian Goodfellow)

генеративно-состязатльные сети (Generative Adversarial Networks)

задачи по повышению разрешения изображений

bicubic - классический алгорим уведичения изображений

с помощью GAN можно сформировать тхлюоаэение сумки по схеме

у мужчины с очкми отнимает мужчину без очков и добавляем лицо женщины образуется лицо женщины с очками.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

from tensorflow.keras.datasets import mnist
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Reshape, Input, BatchNormalization, Dropout
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, LeakyReLU

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# выделим конкретные цифры
x_train = x_train[y_train == 7]
y_train = y_train[y_train == 7]

BUFFER_SIZE = x_train.shape[0]
BATCH_SIZE = 100 # размер выборки кратна 100

BUFFER_SIZE = BUFFER_SIZE // BATCH_SIZE * BATCH_SIZE
x_train = x_train[:BUFFER_SIZE]
y_train = y_train[:BUFFER_SIZE]
print(x_train.shape, y_train.shape)

(6200, 28, 28) (6200,)


In [3]:
x_train = x_train / 255
x_test = x_test / 255

# формируем тензоры для обучающей и тестовой выборки
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

# наборы батчей перемешанных семёрок
train_dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# формирование сетей
hidden_dim = 2

In [4]:
def dropout_and_batch():
    return Dropout(0.3)(BatchNormalization())

In [5]:
# формируем генератор
generator = tf.keras.Sequential([
    Dense(7 * 7 * 256, activation='relu', input_shape=(hidden_dim,)),
    BatchNormalization(),
    Reshape((7, 7, 256)),
    Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', activation='sigmoid'),    
])

In [6]:
# формируем дискриминатор
discriminator = tf.keras.Sequential()
discriminator.add(Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]))
discriminator.add(LeakyReLU())
discriminator.add(Dropout(0.3))

discriminator.add(Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
discriminator.add(LeakyReLU())
discriminator.add(Dropout(0.3))

discriminator.add(Flatten())
discriminator.add(Dense(1))

# потери
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [7]:
def generator_loss(fake_output):
    loss = cross_entropy(tf.ones_like(fake_output), fake_output)
    return loss

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [8]:
# обучение
@tf.function
def train_step(images): # на в ход подаём батч, пакет реальных иображений
    # формируем такой же батч, но нормальных случайных величин, с 0 мат ожиданием и 1 дисперсией
    noise = tf.random.normal([BATCH_SIZE, hidden_dim]) 

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

In [9]:
def train(dataset, epochs):
    history = []
    MAX_PRINT_LABEL = 10
    th = BUFFER_SIZE // (BATCH_SIZE * MAX_PRINT_LABEL)

    for epoch in range(1, epochs +1):
        print(f'{epoch}/{EPOCHS}: ', end='')

        start = time.time()
        n = 0

        gen_loss_epoch = 0
        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch)
            gen_loss_epoch += K.mean(gen_loss)
            if (n % th == 0): print('=', end='')
            n += 1
        
        history += [gen_loss_epoch / n]
        print(': ' + str(history[-1]))
        print('Время эпохи {} составляет {} секунд'.format(epoch, time.time() - start))

    return history

In [10]:
# запуск процесса обучения
EPOCHS = 20
history = train(train_dataset, EPOCHS)

plt.plot(history)
plt.grid(True)
plt.show()

1/20: ===========: tf.Tensor(1.5339868, shape=(), dtype=float32)
Время эпохи 1 составляет 77.23521780967712 секунд
2/20: ===========: tf.Tensor(2.609603, shape=(), dtype=float32)
Время эпохи 2 составляет 76.42991399765015 секунд
3/20: ===========: tf.Tensor(3.26908, shape=(), dtype=float32)
Время эпохи 3 составляет 80.94363045692444 секунд
4/20: ===========: tf.Tensor(3.4889956, shape=(), dtype=float32)
Время эпохи 4 составляет 75.00151205062866 секунд
5/20: ===========: tf.Tensor(3.7302277, shape=(), dtype=float32)
Время эпохи 5 составляет 74.8589518070221 секунд
6/20: ====

KeyboardInterrupt: ignored

In [ ]:
# отображение результатов генерации
n = 2
total = 2 * n + 1

plt.figure(figsize=(total, total))

num = 1
for i in range(-n, n + 1):
    for j in range(-n , n + 1):
        ax = plt.subplot(total, total, num)
        num + = 1
        img = generator.predict(np.expand_dims([0.5 * i / n, 0.5 * j / n], axis=0))
        plt.imshow(img[0, :, :, 0], cmap='gray')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

plt.show()